## TRANSIENCE nomenclature template and Variable Ontology

### Importing Libraries

In [1]:
import pandas as pd
import yaml
from pprint import pprint
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL

### Loading Data

In [2]:
# Load the CSV template
input_df = pd.read_csv('data/transience_nomenclature.csv', 
                 sep=';', encoding='utf-8').fillna('')

### Creating the YAML file

In [3]:
# Prepare the output dict
output = {
    'transience_nomenclature': {
        'variables': [],
        'parameters': {
            'inputs': [],
            'outputs': []
        }
    }
}

# Helper to append entries
def make_entry(row):
    entry = {}
    if row['Definition']:
        entry['description'] = row['Definition']
    if row['Unit']:
        entry['unit'] = row['Unit']
    if row['Output_of_model(s)']:
        entry['output_of_model'] = row['Output_of_model(s)']
    if row['Input_to_model(s)']:
        entry['input_to_model'] = row['Input_to_model(s)']
    if row['Defined by']:
        entry['defined_by'] = row['Defined by']
    return { row['Name']: entry }

# Fill variables
for _, row in input_df[input_df['Type'] == 'variable'].iterrows():
    output['transience_nomenclature']['variables'].append(make_entry(row))

# Fill parameter inputs
for _, row in input_df[input_df['Type'] == 'parameter_input'].iterrows():
    output['transience_nomenclature']['parameters']['inputs'].append(make_entry(row))

# Fill parameter outputs
for _, row in input_df[input_df['Type'] == 'parameter_output'].iterrows():
    output['transience_nomenclature']['parameters']['outputs'].append(make_entry(row))

# Create YAML 
with open('data/transience_nomenclature.yaml', 'w', encoding='utf-8') as f:
    yaml.safe_dump(
        output,
        f,
        sort_keys=False,
        allow_unicode=True,
        width=10000
    )

### Creating Ontology from YAML

In [4]:
with open('data/transience_nomenclature.yaml', 'r') as yaml_file:
    data = yaml.safe_load(yaml_file)

In [5]:
# Extract and format the variables into a more usable Python structure
variables = {}
for item in data['transience_nomenclature']['variables']:
    for var_name, var_info in item.items():
        variables[var_name] = var_info

parameters = {}

# Parameter inputs
for item in data['transience_nomenclature']['parameters']['inputs']:
    for param_name, param_info in item.items():
        parameters[param_name] = param_info

# Parameter outputs
for item in data['transience_nomenclature']['parameters']['outputs']:
    for param_name, param_info in item.items():
        parameters[param_name] = param_info

In [8]:
# RDF graph and namespaces
g = Graph()
EX = Namespace("http://transience.eu/ontologies/")
g.bind("ex", EX)
g.bind("rdfs", RDFS)
g.bind("owl", OWL)

# ontology declaration
ontology_uri = URIRef("http://transience.eu/ontologies/")
g.add((ontology_uri, RDF.type, OWL.Ontology))

# base classes
g.add((EX.Variable, RDF.type, OWL.Class))
g.add((EX.Variable, RDFS.comment, Literal("Base class for all Variables")))

g.add((EX.Parameter, RDF.type, OWL.Class))
g.add((EX.Parameter, RDFS.comment, Literal("Base class for all Parameters")))

g.add((EX.ParameterInput, RDF.type, OWL.Class))
g.add((EX.ParameterInput, RDFS.subClassOf, EX.Parameter))
g.add((EX.ParameterInput, RDFS.comment, Literal("Parameter Input")))

g.add((EX.ParameterOutput, RDF.type, OWL.Class))
g.add((EX.ParameterOutput, RDFS.subClassOf, EX.Parameter))
g.add((EX.ParameterOutput, RDFS.comment, Literal("Parameter Output")))

uri_name = var_name.replace(" ", "_") \
                    .replace("|", "_") \
                    .replace("/", "_") \
                    .replace("[", "") \
                    .replace("]", "")
# variables as classes
for item in data["transience_nomenclature"]["variables"]:
    for var_name, var_info in item.items():
        uri_name = var_name.replace(" ", "_") \
                    .replace("|", "_") \
                    .replace("/", "_") \
                    .replace("[", "") \
                    .replace("]", "")
        var_uri = EX[uri_name]
        g.add((var_uri, RDF.type, OWL.Class))
        g.add((var_uri, RDFS.subClassOf, EX.Variable))
        g.add((var_uri, RDFS.label, Literal(var_name)))
        g.add((var_uri, RDFS.comment, Literal(var_info["description"])))
        g.add((var_uri, EX.unit, Literal(var_info["unit"])))


# parameter inputs as classes
for item in data['transience_nomenclature']['parameters']['inputs']:
    for param_name, param_info in item.items():
        uri_name = param_name.replace(" ", "_") \
                    .replace("|", "_") \
                    .replace("/", "_") \
                    .replace("[", "") \
                    .replace("]", "")
        param_uri = EX[uri_name]
        g.add((param_uri, RDF.type, OWL.Class))
        g.add((param_uri, RDFS.subClassOf, EX.ParameterInput))
        g.add((param_uri, RDFS.label,   Literal(param_name)))
        g.add((param_uri, RDFS.comment, Literal(param_info.get("description", ""))))
        g.add((param_uri, EX.unit,      Literal(param_info.get("unit", ""))))

# parameter outputs as classes
for item in data['transience_nomenclature']['parameters']['outputs']:
    for param_name, param_info in item.items():
        uri_name = param_name.replace(" ", "_") \
                    .replace("|", "_") \
                    .replace("/", "_") \
                    .replace("[", "") \
                    .replace("]", "")
        param_uri = EX[uri_name]
        g.add((param_uri, RDF.type, OWL.Class))
        g.add((param_uri, RDFS.subClassOf, EX.ParameterOutput))
        g.add((param_uri, RDFS.label,   Literal(param_name)))
        g.add((param_uri, RDFS.comment, Literal(param_info.get("description", ""))))
        g.add((param_uri, EX.unit,      Literal(param_info.get("unit", ""))))

# Save to file
g.serialize(destination="data/transience_variables.owl", format="xml")

<Graph identifier=Nac787108d9a5436cba6fc310f903c229 (<class 'rdflib.graph.Graph'>)>